## Septum volume linearisation

Rough notebook to work out the full form of Jallon's linearisation of $V_{spt}$. Monkey-patches PressureVolume to work with SymPy instead of PyTorch

In [1]:
import sympy as sp

v_spt, v_lv, v_rv, e_t = sp.symbols('v_spt v_lv v_rv e_t')

In [2]:
from biophysical_models import base_classes

base_classes.torch = sp

class PressureVolume(base_classes.PressureVolume):
    def __init__(self, name: str):
        self.e_es, self.v_d, self.v_0, self.lam, self.p_0 = sp.symbols(f'{name}.e_es {name}.v_d {name}.v_0 {name}.lam {name}.p_0')

spt = PressureVolume('spt')
lvf = PressureVolume('lvf')
rvf = PressureVolume('rvf')

In [3]:
v_lvf = v_lv - v_spt
v_rvf = v_rv + v_spt

p_lvf = e_t * lvf.p_es(v_lvf) + (1 - e_t) * lvf.p_ed_linear(v_lvf)
p_rvf = e_t * rvf.p_es(v_rvf) + (1 - e_t) * rvf.p_ed_linear(v_rvf)
p_spt = e_t * spt.p_es(v_spt) + (1 - e_t) * spt.p_ed_linear(v_spt)

p_spt_rhs = p_lvf - p_rvf


In [4]:
eq = sp.Eq(p_spt, p_spt_rhs)
eq.simplify()

Eq(e_t*spt.e_es*(spt.v_d - v_spt) - spt.lam*spt.p_0*(e_t - 1)*(spt.v_0 - v_spt), e_t*lvf.e_es*(lvf.v_d - v_lv + v_spt) + e_t*rvf.e_es*(-rvf.v_d + v_rv + v_spt) - lvf.lam*lvf.p_0*(e_t - 1)*(lvf.v_0 - v_lv + v_spt) - rvf.lam*rvf.p_0*(e_t - 1)*(-rvf.v_0 + v_rv + v_spt))

Solve for $V_{spt}$ using SymPy:

In [5]:
sol = sp.solve(eq, v_spt)[0]
sol

(-e_t*lvf.e_es*lvf.v_d + e_t*lvf.e_es*v_lv + e_t*lvf.lam*lvf.p_0*lvf.v_0 - e_t*lvf.lam*lvf.p_0*v_lv + e_t*rvf.e_es*rvf.v_d - e_t*rvf.e_es*v_rv - e_t*rvf.lam*rvf.p_0*rvf.v_0 + e_t*rvf.lam*rvf.p_0*v_rv + e_t*spt.e_es*spt.v_d - e_t*spt.lam*spt.p_0*spt.v_0 - lvf.lam*lvf.p_0*lvf.v_0 + lvf.lam*lvf.p_0*v_lv + rvf.lam*rvf.p_0*rvf.v_0 - rvf.lam*rvf.p_0*v_rv + spt.lam*spt.p_0*spt.v_0)/(e_t*lvf.e_es - e_t*lvf.lam*lvf.p_0 + e_t*rvf.e_es - e_t*rvf.lam*rvf.p_0 + e_t*spt.e_es - e_t*spt.lam*spt.p_0 + lvf.lam*lvf.p_0 + rvf.lam*rvf.p_0 + spt.lam*spt.p_0)

Try and simplify it a bit:

In [6]:
sol.factor(lvf.e_es, lvf.p_0, lvf.lam, lvf.v_0, rvf.e_es, rvf.p_0, rvf.lam, rvf.v_0, spt.e_es, spt.p_0, spt.lam, spt.v_0, v_lv, v_rv)

(-e_t*lvf.e_es*lvf.v_d + e_t*lvf.e_es*v_lv + e_t*rvf.e_es*rvf.v_d - e_t*rvf.e_es*v_rv + e_t*spt.e_es*spt.v_d + lvf.lam*lvf.p_0*lvf.v_0*(e_t - 1) + lvf.lam*lvf.p_0*v_lv*(1 - e_t) + rvf.lam*rvf.p_0*rvf.v_0*(1 - e_t) + rvf.lam*rvf.p_0*v_rv*(e_t - 1) + spt.lam*spt.p_0*spt.v_0*(1 - e_t))/(e_t*lvf.e_es + e_t*rvf.e_es + e_t*spt.e_es + lvf.lam*lvf.p_0*(1 - e_t) + rvf.lam*rvf.p_0*(1 - e_t) + spt.lam*spt.p_0*(1 - e_t))

Manually simplify instead:

In [7]:
num = e_t * (lvf.p_es(v_lv) - rvf.p_es(v_rv) + spt.e_es * spt.v_d) + (1 - e_t) * (lvf.p_ed_linear(v_lv) - rvf.p_ed_linear(v_rv) + spt.lam * spt.p_0 * spt.v_0)
den = e_t * (lvf.e_es + rvf.e_es + spt.e_es) + (1 - e_t) * (lvf.lam * lvf.p_0 + rvf.lam * rvf.p_0 + spt.lam * spt.p_0)
sol2 = num/den

Check that the proposed form of the solution is equal to SymPy's solution:

In [8]:
sp.simplify(sol - sol2)

0